# **NDSL Orchestration Basics**

### **Introduction**

When writing code using NDSL, there will be moments where an algorithm or code pattern does not match the stencil paradigm, and shoehorning the algorithm into the paradigm increases development difficulty.  For these moments, we have a capability called orchestration that enables developers to use regular Python for non-stencil algorithms alongside stencil-based code via [DaCe](https://github.com/spcl/dace).  DaCe also will attempt to find optimizations before output C++ code.

In this example, we will explore how to orchestrate a codebase using NDSL.

### **Orchestration Example**

We'll step through a simple example that will orchestrate a codebase containing stencils and Python code.  First we'll import the necessary packages.

In [ ]:
import numpy as np
from gt4py.cartesian.gtscript import (
    PARALLEL,
    computation,
    interval,
)
from ndsl import (
    StencilFactory,
    DaceConfig,
    orchestrate,
    QuantityFactory,
)
from ndsl.constants import X_DIM, Y_DIM, Z_DIM
from ndsl.dsl.typing import FloatField, Float
from ndsl.boilerplate import get_factories_single_tile_orchestrated

Next we'll define a simple stencil that sums the values around a point and applies a weight factor to that sum.  Note that unlike [previous](./01_gt4py_basics.ipynb#Copy_Stencil_example) examples, we are not using the `@stencil` decorator since this stencil will be referenced within a `StencilFactory` function call.

In [ ]:
def localsum_stencil(
    field: FloatField,   # type: ignore
    result: FloatField,  # type: ignore
    weight: Float,       # type: ignore
):
    with computation(PARALLEL), interval(...):
        result = weight * (
            field[1, 0, 0] + field[0, 1, 0] + field[-1, 0, 0] + field[0, -1, 0]
        )

We'll define an object that enables the orchestration and combines both stencils and regular Python codes.  The orchestration occurs with the `orchestrate` call in the `__init__` definition.  Within `__call__`, there's a combination of both stencil and regular python codes.

In [ ]:
class LocalSum:
    def __init__(
        self, stencil_factory: StencilFactory, quantity_factory: QuantityFactory
    ) -> None:
        orchestrate(
            obj=self,
            config=stencil_factory.config.dace_config
            or DaceConfig(None, stencil_factory.backend),
        )
        grid_indexing = stencil_factory.grid_indexing
        self._local_sum = stencil_factory.from_origin_domain(
            localsum_stencil,  # <-- gt4py stencil function wrapped into NDSL
            origin=grid_indexing.origin_compute(),
            domain=grid_indexing.domain_compute(),
        )
        self._tmp_field = quantity_factory.zeros(
            [X_DIM, Y_DIM, Z_DIM], "n/a", dtype=dtype
        )
        self._n_halo = quantity_factory.sizer.n_halo

    def __call__(self, in_field: FloatField, out_result: FloatField) -> None:
        self._local_sum(in_field, out_result, 2.0)  # GT4Py Stencil
        tmp_field = out_result[:, :, :] + 2         # Regular Python code
        self._local_sum(tmp_field, out_result, 2.0) # GT4Py Stencil

Next, we'll create a simple driver that defines the domain and halo size, specifies the backend (`dace:cpu` in order to use DaCe), and uses the boilerplate code to create a stencil and quantity factory objects.  These objects help define the computational domain used for this particular example.  After defining quantities (`in_field` and `out_field`) to hold the appropriate values and creating an object `local_sum` for our combined stencil/Python calculation, `local_sum` is called to perform the computation.  In the output, we can see DaCe orchestrating the code. 

In [ ]:
# ----- Driver ----- #

if __name__ == "__main__":
    # Settings
    dtype = np.float64
    origin = (0, 0, 0)
    rebuild = True
    tile_size = (3, 3, 3)

    # Setup
    stencil_factory, qty_factory = get_factories_single_tile_orchestrated(
        nx=tile_size[0],
        ny=tile_size[1],
        nz=tile_size[2],
        nhalo=2,
    )
    local_sum = LocalSum(stencil_factory, qty_factory)

    in_field = qty_factory.zeros([X_DIM, Y_DIM, Z_DIM], "n/a", dtype=dtype)
    in_field.view[:] = 2.0
    out_field = qty_factory.zeros([X_DIM, Y_DIM, Z_DIM], "n/a", dtype=dtype)

    # Run
    local_sum(in_field, out_field)